# Referenzdatensatz erstellen

In [44]:
# Besonderheit HASOC 2020 Daten: Zeilenumbrüche innerhalb der Tweets
# --> notwendig, die Dateien erst so aufzubereiten, dass nur ein Tweet pro Zeile steht

# HASOC 2020 Train: "..\Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new.txt"
# HASOC 2020 Test: "..\Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new.txt"

def rem_white(filename):
    with open(filename, mode="r", encoding="utf-16") as f:
        content = f.readlines()
        content = content[1:] # Erklärungszeile ignorieren
        newcont = []
        for i in range(len(content)):
            # Fall 1: Zeile ist komplett
            if len(content[i].split("\t")) == 5:
                newcont.append(content[i])
            # Fall 2: Zeile ist nicht komplett
            else:
                # letzter Teil einer Zeile erreicht
                if "hasoc_2020_de_" in content[i]:
                    comp_line += content[i]
                    comp_line = comp_line.replace("\n"," ")
                    comp_line += "\n"
                    newcont.append(comp_line)
                # erster Teil einer Zeile 
                elif content[i].startswith("11") :
                    comp_line = content[i]
                # mittlerer Teil einer Zeile, manchmal nur \n
                else: comp_line += content[i]
    nwfilename = filename[:len(filename)-4] + "_formatted" + ".txt"
    with open(nwfilename, mode="w", encoding="utf-8") as outfile:
        for line in newcont:
            outfile.write(line)
    return True

# Bereits formatiert, nicht nochmals durchführen
#rem_white("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new.txt")
#rem_white("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new.txt")

True

In [88]:
# Besonderheit des Covid19-Abusive-Datensatzes: noch nicht unterteilt in Trainings- und Testdaten

from sklearn.model_selection import train_test_split

with open("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset.csv", mode="r", encoding="utf-8") as infile:
    content = infile.readlines()
    explanation = content[0].split("\t")
    explanation = "\t".join([explanation[0],explanation[1],explanation[2],explanation[3]]) +"\n"
    content = content[1:]
    sep_content = [entry.split("\t") for entry in content]
    tweets = [(entry[0], entry[1], entry[2]) for entry in sep_content]
    labels = [entry[3] for entry in sep_content]

    # Separate into train/test
    # insg. 4960 Tweets, laut Paper unterteilt in Trainings-Set mit 3485 Tweets (70%), Validations-Set mit 735, und Test-Set mit 740 Tweets
    # hier entsprechend der Split 70/30, Validation-Daten werden also im Zweifelsfall von dem Testset abgespalten
    traintweets, testtweets, trainlabels, testlabels = train_test_split(tweets, labels, test_size=0.3, train_size=0.7, random_state=4, stratify=labels)

    # Reassemble into filewritable content
    train = []
    test = []
    for i, tweet in enumerate(traintweets):
        train.append("\t".join([tweet[0],tweet[1],tweet[2],trainlabels[i]]))
    for j, tweet in enumerate(testtweets):
        test.append("\t".join([tweet[0],tweet[1],tweet[2],testlabels[j]]))

with open("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_train.txt", mode="w", encoding="utf-8") as trainout:
    trainout.write(explanation)
    for train_point in train:
        trainout.write(train_point+"\n")

with open("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_test.txt", mode="w", encoding="utf-8") as testout:
    testout.write(explanation)
    for test_point in test:
        testout.write(test_point+"\n")

In [24]:
# Besonderheit des GermEval2019-Datensatzes: abgeschnittene erste Zeilen in den Testdaten
# --> notwendig, die Dateien aufzubereiten

# fehlende Buchstaben am Anfang von Tweets: abgeschnitten von GermEval2019 intern beim Übergang der Daten von
# "Testdata_Subtask12" zu "GoldLabelSubtask12"; Zeilennr. sind die gleichen
# z.B. für Zeile 341:
# "enschen, die etwas auf eBay-Kleinanzeigen verticken, wohnen entweder in einer dickichten Gartenlaube oder im obersten Stock Altbau - Hinterhaus. #issso	OTHER	OTHER"


# Testdaten: mit den Originaltweets aus der Datei "germeval2019_Testdata_Subtask12.txt" (dort ohne Labels) speichern

with open("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2.txt", mode="r", encoding="utf-8") as in_test:
    cont_test = in_test.readlines()
    sep_cont = [line.strip().split("\t") for line in cont_test]

# Tweets ohne abgeschnitte Anfänge einlesen
with open("..\Korpora\GermEval-2019-Data\germeval2019_Testdata_Subtask12.txt", mode="r", encoding="utf-8") as in_tweets:
    tweets = in_tweets.readlines()
    tweets = [tweet.strip() for tweet in tweets]

# Tweets mit den Labels zusammenführen
tweets_replaced = [(tweets[i],sep_cont[i][1],sep_cont[i][2]) for i in range(len(sep_cont))]

with open("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2_ersetzt.txt", mode="w", encoding="utf-8") as out_test:
    for line in tweets_replaced:
        out_test.write("\t".join(line)+"\n")

In [45]:
import re

def convert_to_refcorp(filename, corp_id, mod):
    """
    GermEval-Daten, HASOC-Daten und die Daten des Covid19-Abusive-Datensatzes in ein einheitliches Format zu übertragen.
    Input: Datei mit Tabstopp-getrennten Werten (ob txt, csv oder tsv), Korpus-ID, train/test-Information
    Output: Liste von Tupeln à (Referenzkorpus-ID, Tweet, Label1, Label2)
            - ReferenzkorpusID - setzt sich zusammen aus der Korpus-ID,
                                 md_id = "11", falls es um Trainingsdaten (mod=train), "22", falls es um Testdaten (mod=test) geht
                                 und der Zeilennummer in der Ursprungsdatei;
                                 also z.B.: "01220034" - für einen Tweet der Zeile 34, aus den Testdaten des GermEval2018-Datensatzes
            - Tweet            - String des Tweets, URLs sind mit der generischen Twitter-URL "https://t.co" ersetzt
            - Label1           -
            - Label2           -
    """
    newcorp = []
    with open(filename, mode="r", encoding="utf-8") as f:
        text = f.readlines()
        
        # erste Zeile ignorieren bei Covid19 ("05") und HASOC2019 ("03")
        if corp_id == "05" or corp_id == "03": text = text[1:]

        # Bestimmen, welche Formatierungsfunktion genutzt wird
        if corp_id == "01" or corp_id == "02": form_func = format_germeval
        elif corp_id == "03" or corp_id == "04": form_func = format_hasoc
        else: form_func = format_covidabusive

        url_pattern = re.compile('https:\/\/.*?(?: |$)')

        for num, entry in enumerate(text):
            entry = entry.strip()
            tag1, tag2 = "NOT", "NOT"

            tweet, tag1, tag2 = form_func(entry, tag1, tag2)

            # URLs mit generischer Twitter-URL ersetzen
            tweet = url_pattern.sub("https://t.co ", tweet)
            tweet = tweet.strip()

            # Tweet von HTML-Resten entfernen und Emoji-Codierung mit Emojis ersetzen
            tweet = clean_tweet(tweet)

            # gedoppelte und überflüssige Anführungszeichen entfernen
            tweet = tweet.replace('""',"'")
            tweet = tweet.strip('"')

            # ID erstellen
            if mod == "train": md_id = "11"
            elif mod =="test": md_id = "22"
            id_num = f'{num+1:04d}'
            tweet_id = str(corp_id) + str(md_id) + str(id_num)
            
            # der neuen Sammlung hinzufügen
            newcorp.append((tweet_id, tweet, tag1, tag2))
    return newcorp

def format_germeval(entry, tag1, tag2):
    tweet, label1, label2 = entry.split("\t")
    if label1 == "OFFENSE": tag1 = "NEG"
    if label2 == "INSULT": tag2 = "INSOFF"
    elif label2 == "PROFANITY": tag2 = "PRFN"
    elif label2 == "ABUSE": tag2 = "HATE"
    tweet = tweet.replace("|LBR|", " ")
    return tweet, tag1, tag2

def format_covidabusive(entry, tag1, tag2):
    sep = entry.split("\t")
    tweet, l1 = sep[1], sep[3]
    tag2 = "NAN"
    if l1 == "abusive": tag1 = "NEG"
    return tweet, tag1, tag2

def format_hasoc(entry, tag1, tag2):
    sep = entry.split("\t")
    tweet, l1, l2 = sep[1], sep[2], sep[3]
    if l1 == "HOF": tag1 = "NEG"
    if l2 not in ["HATE", "OFFN", "PRFN"] and l1!="NOT": print(l2)
    if l2 == "HATE": tag2 = "HATE"
    elif l2 == "OFFN": tag2 = "INSOFF"
    elif l2 == "PRFN": tag2 = "PRFN"
    return tweet, tag1, tag2


def clean_tweet(tweet):
    """Emojis finden und ersetzen (gefunden: https://stackoverflow.com/questions/67507017/replace-unicode-code-point-with-actual-character-using-regex)
        und HTML-Reste entfernen.
    """
    cleaned = tweet
    # Emojis, die als Text, z.B. "<U+0001F60A>", gespeichert sind: als utf-8 formatieren
    cleaned = re.sub(r'<U\+([A-F0-9]+)>', lambda x: chr(int(x.group(1), 16)), cleaned)
    cleaned = re.sub(r"&lt;" , "<", cleaned)	 
    cleaned = re.sub(r"&gt;" , ">", cleaned)    
    cleaned = re.sub(r"&amp;" , "&", cleaned)
    cleaned = re.sub(r'\"', '"', cleaned)
    cleaned = re.sub(r'\""', '"', cleaned)
    return cleaned

In [46]:
# GermEval2018
germeval2018train_converted = convert_to_refcorp("..\Korpora\GermEval-2018-Data-master\germeval2018.training.txt", "01", "train")
germeval2018test_converted = convert_to_refcorp("..\Korpora\GermEval-2018-Data-master\germeval2018.test.txt", "01", "test")

# GermEval2019
germeval2019train_converted = convert_to_refcorp("..\Korpora\GermEval-2019-Data\germeval2019.training_subtask1_2_korrigiert.txt", "02", "train")
germeval2019test_converted = convert_to_refcorp("..\Korpora\GermEval-2019-Data\germeval2019GoldLabelsSubtask1_2_ersetzt.txt", "02", "test")

# HASOC 2019
hasoc2019train_converted = convert_to_refcorp("..\Korpora\german_dataset_hasoc2019\german_dataset\german_dataset.tsv", "03", "train")
hasoc2019test_converted = convert_to_refcorp("..\Korpora\german_dataset_hasoc2019\german_dataset\hasoc_de_test_gold.tsv", "03", "test")

# HASOC 2020
hasoc2020train_converted = convert_to_refcorp("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_train_new_formatted.txt", "04", "train")
hasoc2020test_converted = convert_to_refcorp("..\Korpora\German_2020_hasoc\German\hasoc_2020_de_test_new_formatted.txt", "04", "test")

# Covid19 Abusive
covidabusivetrain_converted = convert_to_refcorp("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_train.txt", "05", "train")
covidabusivetest_converted = convert_to_refcorp("..\Korpora\german-abusive-language-covid-19-main\covid_2021_dataset_test.txt", "05", "test")

In [49]:
# Referenzdatensatz zusammenstellen
# Bereits ausgeführt, nicht noch einmal ausführen

refcorp_train = germeval2018train_converted + germeval2019train_converted + hasoc2019train_converted + hasoc2020train_converted + covidabusivetrain_converted
refcorp_test = germeval2018test_converted + germeval2019test_converted + hasoc2019test_converted + hasoc2020test_converted + covidabusivetest_converted

# refcorp_gsmt = refcorp_train + refcorp_test # insg. 28596 Tweets
# refcorp_gsmt_HATE = [entry for entry in refcorp_gsmt if entry[3] == "HATE"] # insg. 3060 Tweets

import random

random.shuffle(refcorp_train)
random.shuffle(refcorp_test)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train.txt", mode="w", encoding="utf-8") as reftrainout:
    reftrainout.write("corpus_id\ttweet\tbinarylabel\tfinelabel\n")
    for reftweet in refcorp_train:
        reftrainout.write("\t".join(reftweet)+"\n")
    
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test.txt", mode="w", encoding="utf-8") as reftestout:
    reftestout.write("corpus_id\ttweet\tbinarylabel\tfinelabel\n")
    for reftweet in refcorp_test:
        reftestout.write("\t".join(reftweet)+"\n")

1. Anonymisierung der @user-Erwähnungen zu "@user
2. Duplikate entfernen (bei Paaren sehr ähnlicher Tweets einen entfernen)
3. Mit den verschiedenen Annotationsebenen- und detaills separat speichern

##### 1. Anonymisierung der @user-Erwähnungen

In [81]:
import re

def anonym_atuser(tweet):
    tweet_anonym = re.sub('@[^@ ]+?@', '@user@', tweet)
    tweet_anonym = re.sub('@[^@ ]+? ', '@user ', tweet_anonym)
    tweet_anonym = re.sub('@[^@ ]+?$', '@user', tweet_anonym)
    return tweet_anonym

# von '@rspctfl@houelle_beck @ergroovt @ThomasMichael71 @ksemann2 @DrKassandraPari Dann hättest Du nichts dagegen, wenn ein Lehrer ein Zeichen d PKK im Unterricht trägt? Oder die Flagge Israels? Oder ein Anstecker der AfD? @Innenwelttramp'
# zu '@user@user @user @user @user @user Dann hättest Du nichts dagegen, wenn ein Lehrer ein Zeichen d PKK im Unterricht trägt? Oder die Flagge Israels? Oder ein Anstecker der AfD? @user'

##### 2. Duplikate entfernen

In [ ]:
# Jaccard-Index-Funktionen zur Berechnung der Ähnlichkeit

def make_bag(string1):
    """ String als Multiset speichern; 
        In Form eines dicts: jedes vorkommende Unigram als Key, die Anzahl der Nutzungen als Value
        Als zusätzlichen Wert auch die Länge des Strings speichern
    """
    stringbag = dict()
    stringset = set(string1)
    for ch1 in stringset:
        stringbag[ch1] = string1.count(ch1)
    stringbag["len"] = len(string1)
    stringbag["tok"] = set()
    stringbag["tok"] = set(stringbag.keys()) - {"len", "tok"}
    return stringbag

def jaccard(set1, set2):
    """Jaccard-Index von zwei Sets (von zwei Strings) berechnen.
       Max.-Wert: 1 (Gleichheit)
    """
    jaccard_index = len(set1 & set2) / len(set1 | set2)
    return jaccard_index

def jaccard_multisets(bag1, bag2):
    """Den Jaccard-Index für zwei Multisets (von zwei Strings) berechnen.
       Max.-Wert: 0.5
    """
    # Schnittmenge bauen, die Gesamtlänge speichern
    schnitt_len = 0
    schnitt = bag1["tok"] & bag2["tok"]
    for gram in schnitt:
        l = min(bag1[gram],bag2[gram])
        schnitt_len += l

    # Länge der Vereinigung ermitteln
    vereinigung_len = bag1["len"] + bag2["len"]
    
    jaccard_index = schnitt_len / vereinigung_len
    return jaccard_index

In [ ]:
def calculate_bags(corpus):
    """Multisets (bags) für alle Tweets in einem Korpus berechen.
    Input: Korpus
    Output: Liste von Multisets (Index in der Liste == Index in der Korpusliste)
    """
    bags = [make_bag(tweet[1]) for tweet in corpus]
    return bags

def calculate_sets(corpus):
    """Set für alle Tweets in einem Korpus berechnen; als Liste der Sets speichern.
    """
    sets = [set(tweet[1]) for tweet in corpus]
    return sets

def collect_duplicates(corpus, sets, simfunc, cutoff):
    """ Duplikate sammeln
    Input: Korpus, Liste von (Multi)Sets für alle Tweets im Korpus, Vergleichsfunktion, Ähnlichkeitsgrenzwert
    Output: Liste potentieller Duplikate (als Tupel der Korpuseinträge)
    """
    duplicates = []
    for i in range(len(corpus)):
        # if i % 100 == 0: print("i",i) # Zur Zeitmessung
        for j in range(len(corpus)):
            # Kein Vergleich mit sich selbst oder bereits in die andere Richtung verglichener Paare
            if i >= j: continue
            else:
                jacc = simfunc(sets[i],sets[j])
                if jacc >= cutoff: # z.B. 0.47 für multisets
                    duplicates.append((corpus[i],corpus[j]))
    return duplicates

In [ ]:
with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train.txt", mode="r", encoding="utf-8") as in_train:
    train = in_train.readlines()

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test.txt", mode="r", encoding="utf-8") as in_test:
    test = in_test.readlines()

total = train[1:] + test[1:] # ohne Erklärungszeile
total = [entry.strip().split("\t") for entry in total]


# @user-Erwähnungen anonymisieren
total_anonym = [(entry[0],anonym_atuser(entry[1]),entry[2],entry[3]) for entry in total]

# Sets aller Tweets berechnen
tweet_sets = calculate_sets(total_anonym)

#### Jaccard-Index berechnen

In [ ]:
bags = calculate_bags(total_anonym) # Multimenge für alle Tweets berechnen

In [ ]:
# Jaccard-Index berechnen
dups_multisets = collect_duplicates(total_anonym, bags, jaccard_multisets, 0.47)
# Berechnung (anfangs): ca. 30 Sekunden/200 Tweets --> insg. ca 70 Minuten
# Aber: Wird mit der Zeit schneller (da nach und nach bereits alle Vergleiche in eine Richtung bereits geschehen) --> insg.: 42m 9.9s

#import json
#dups_json = json.dumps(dups_multisets)
#with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\Duplikat_IDs.json", mode="w", encoding="utf-8") as out_dups:
#    out_dups.write(dups_json)

In [ ]:
# Duplikat-Cluster bestimmen, um Kettenvergleiche zu vermeiden
# --> falls ID1 ähnlich ID2: zusammen in ein Set, und jede weitere ID, die einer der beiden IDs ähnlich ist dazu

cluster = []
ID_cluster_ref = dict()
for duplicate_tup in dups_multisets:
    tweet1, tweet2 = duplicate_tup
    ID1, ID2 = tweet1[0], tweet2[0]
    # 1. weder ID1 noch ID2 vorhanden
    if (ID1 not in ID_cluster_ref) and (ID2 not in ID_cluster_ref):
        cluster_num = len(cluster)
        ID_cluster_ref[ID1], ID_cluster_ref[ID2] = cluster_num, cluster_num
        cluster.append({tweet1, tweet2})
    # 2. eine von beiden vorhanden --> beide IDs ins vorhandene Cluster integrieren
    elif ID1 in ID_cluster_ref:
        cluster_num = ID_cluster_ref[ID1]
        ID_cluster_ref[ID2] = cluster_num
        cluster[cluster_num].add(tweet2)
    elif ID2 in ID_cluster_ref:
        cluster_num = ID_cluster_ref[ID2]
        ID_cluster_ref[ID1] = cluster_num
        cluster[cluster_num].add(tweet1)
    # 3. Beide bereits vorhanden
    else:
        assert ID_cluster_ref[ID1] == ID_cluster_ref[ID2]

# schwarze Liste zu löschender IDs
schwarzeListe = []
for gruppe in cluster[2:]:
    grp = list(gruppe)
    for tweet in grp[1:]:
        schwarzeListe.append(tweet[0])


In [ ]:
print(len(ID_cluster_ref)) # 321 Tweets, die einander in irgendeiner Konstellation ähnlich sind
print(len(cluster)) # 100 Cluster
print(cluster[0]) 
print(len(schwarzeListe)) # 223 zu löschende Tweets

# Cluster 0: vergessene erste Zeile (Erklärung) --> TODO nochmal laufen lassen, json neu speichern
# Cluster 1: Ähnlichkeit durch wiederholte @user Erwähnungen, Text selbst nicht ähnlich
# tatsächlich ähnlich, nur einen der Tweets behalten (Zufall): Clusters 2 - 8 
# Cluster 4: 04110200 und zwei weitere HASOC2020 Tweets: "https:/…" als Rest -> mit generischem Link ersetzten

In [ ]:
# Ohne die Duplikate in neue Dateien schreiben (nicht mehr anonymisiert)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Train_OD.txt", mode="w", encoding="utf-8") as out_train:
    for tweet in train:
        if tweet[0] not in schwarzeListe:
            out_train.write(tweet)

with open("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\RefKorpHateSpeechDe_Test_OD.txt", mode="w", encoding="utf-8") as out_test:
    for tweet in test:
        if tweet[0] not in schwarzeListe:
            out_test.write(tweet)